# Make Pokemon SV Datasets

In [1]:
import json
import pprint
from PIL import Image, ImageFilter
from PIL import ImageDraw

import glob
import re
import os
from loguru import logger
from tqdm import tqdm
import numpy as np

import cv2
import sys
import shutil

## Setting param

データセットのパスや動画のフォルダなどのパラメーターを設定します．

In [87]:
capture_dir = "capture"
capture_video_dir = "video"
capture_image_dir = "image"

capture_video_path = capture_dir + "/" + capture_video_dir
capture_image_path = capture_dir + "/" + capture_image_dir

root_path = "/home"

#diff_image_th = 1000000
diff_image_th = 760000000
save_freq     = 100

datasets_dir  = "datasets"
datasets_ver  = "v1.0"
datasets_path = datasets_dir + "/" + datasets_ver

anotate_full = datasets_path + "/result.json"
anotate_full_repath = datasets_path + "/result_repath.json"

anotate_train_name = "pokemon_sv_train.json"
anotate_train_path = datasets_path + "/" + anotate_train_name
anotate_valid_name = "pokemon_sv_valid.json"
anotate_valid_path = datasets_path + "/" + anotate_valid_name

image_full_dir  = "images"
image_train_dir = "train2017"
image_valid_dir = "val2017"

In [86]:
%cd $root_path

/home


In [73]:
!ls

Dockerfile  README.md  capture	datasets  docker-compose.yml  notebook	utils


## キャプチャー動画の分解

キャプチャーした動画を分解して画像に変換します．

変換のない静止した状態の画像はスキップした上で，`save_freq`フレームごとに画像を保存します．


### キャプチャー動画のリストを取得

In [74]:
glob_path = capture_video_path + "/*.mp4"
video_list = glob.glob(glob_path, recursive=True)
pprint.pprint(video_list)

['capture/video/2022-11-19 16-34-10.mp4']


### 動画の分解と保存

In [76]:
def analysis_video(video_path):
    
    video_name = video_path.split("/")[-1]
    video_single_path = capture_image_path + "/" + video_name
    logger.info("{:>20} : {}".format("video_single_path", video_single_path))
    os.makedirs(video_single_path, exist_ok=True)
       
    
    cap = cv2.VideoCapture(video_path)
    total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    count = 0
    image_id = 1
    
    for _ in tqdm(range(total_frame_count)):
    #while True:
        ret, frame = cap.read()

        # 読み込み可能かどうか判定
        if ret:
            #logger.info("========================")
            #logger.info("{:>20} : {}".format("count", count))
            
            # 0番目は pre frameに登録のみで処理はskip
            if(count==0):
                pre_frame = frame
            else:
                # 0番目以降は処理
                
                if(image_id % save_freq == 0):

                    # 差分を計算
                    diff_image = np.sum(np.abs(pre_frame - frame))
                    #logger.info("{:>20} : {}".format("diff_image", diff_image))

                    # 閾値以上なら処理する
                    if(diff_image > diff_image_th):
                        # 一定間隔で画像を保存
                        if(image_id % save_freq == 0):
                            save_image_name = "{:09d}.jpg".format(image_id)
                            save_image_path = video_single_path + "/" + save_image_name
                            #logger.info("{:>20} : {}".format("save_image_path", save_image_path))
                            cv2.imwrite(save_image_path, frame)


                    pre_frame = frame
                
                image_id += 1
                
                
            count += 1
        else:
            logger.info("Video Fin ...")
            break
            
        

In [77]:
def video_section():
    for video_path in video_list:
        logger.info("{:>20} : {}".format("video_path", video_path))
        analysis_video(video_path)

In [78]:
video_section()

2022-11-21 04:22:12.957 | INFO     | __main__:video_section:3 -           video_path : capture/video/2022-11-19 16-34-10.mp4
2022-11-21 04:22:12.960 | INFO     | __main__:analysis_video:5 -    video_single_path : capture/image/2022-11-19 16-34-10.mp4
100%|█████████▉| 76428/76429 [05:43<00:00, 222.63it/s]


## 画像をアノテーション

## アノテーションファイルの修正

exportされたアノテーションファイル`datasets\v0\result.json`は画像のパスが`COCO`フォーマットになっていないので修正します．



anotate_full読み込みます

In [89]:
anotate_full

'datasets/v1.0/result.json'

In [90]:
with open(anotate_full, 'rt', encoding='UTF-8') as annotations:
    result_coco = json.load(annotations)

パスを修正しファイル名にします．

In [91]:
for i in range(len(result_coco["images"])):
    file_name = result_coco["images"][i]['file_name']    
    result_coco["images"][i]['file_name'] = file_name.split("/")[-1]

書き出します．

In [92]:
with open(anotate_full_repath, 'wt', encoding='UTF-8') as coco:
        json.dump(result_coco, coco, indent=2, sort_keys=True)

In [93]:
anotate_full_repath

'datasets/v1.0/result_repath.json'

## データセットの split

データセットの分割します．

In [94]:
!python utils/cocosplit.py --having-annotations --multi-class -s 0.8 $anotate_full_repath $anotate_train_path $anotate_valid_path

Saved 625 entries in datasets/v1.0/pokemon_sv_train.json and 157 in datasets/v1.0/pokemon_sv_valid.json


In [100]:
def move_datasets_image_file(target_dir, anno_path):
    
    logger.info("{:>20} : {}".format("target_dir", target_dir))
    logger.info("{:>20} : {}".format("anno_path", anno_path))
    os.makedirs(target_dir, exist_ok=True)
    
    with open(anno_path, 'rt', encoding='UTF-8') as annotations:
        result_coco = json.load(annotations)

    for i in tqdm(range(len(result_coco["images"]))):
        #logger.info(">>>>>>>>>>>> {:>20} : {}".format("i", i))
        
        file_name = result_coco["images"][i]['file_name']   
        #logger.info("{:>20} : {}".format("file_name", file_name))
        
        source_path =  datasets_path + "/" + image_full_dir + "/" + file_name
        #logger.info("{:>20} : {}".format("source_path", source_path))
        
        target_path =  target_dir + "/" + file_name
        #logger.info("{:>20} : {}".format("target_path", target_path))
        
        shutil.copyfile(source_path, target_path)
        
    #pprint.pprint(result_coco)

In [101]:
move_datasets_image_file(target_dir=datasets_path + "/" + image_train_dir, anno_path=anotate_train_path)

2022-11-21 06:13:47.094 | INFO     | __main__:move_datasets_image_file:3 -           target_dir : datasets/v1.0/train2017
2022-11-21 06:13:47.096 | INFO     | __main__:move_datasets_image_file:4 -            anno_path : datasets/v1.0/pokemon_sv_train.json
100%|██████████| 268/268 [00:26<00:00, 10.23it/s]


In [102]:
move_datasets_image_file(target_dir=datasets_path + "/" + image_valid_dir, anno_path=anotate_valid_path)

2022-11-21 06:14:13.344 | INFO     | __main__:move_datasets_image_file:3 -           target_dir : datasets/v1.0/val2017
2022-11-21 06:14:13.346 | INFO     | __main__:move_datasets_image_file:4 -            anno_path : datasets/v1.0/pokemon_sv_valid.json
100%|██████████| 118/118 [00:11<00:00, 10.61it/s]
